# Защита персональных данных клиентов / Protection of clients' personal data

В данном проекте мы разработаем метод защиты персональных данных клиентов страховой компании. Важная часть проекта будет состоять в преобразовании данных таким образом, чтобы метрика модели машинного обучения не ухудшилась.

----

In this project, we will develop a method for protecting personal data of clients of an insurance company. An important part of the project will be to transform the data in such a way that the machine learning model's metric does not deteriorate.

## Загрузка данных / Data import

In [1]:
import pandas as pd
import numpy as np
import dtale
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate

In [2]:
try:
    data = pd.read_csv('/home/uladzislau/Documents/Yandex DS/Projects/Linear Algebra/insurance.csv')
except:
    data = pd.read_csv('/Users/vlad/Documents/Documents Linux/Yandex DS/Projects/YandexPracticum-projects/data_prot/insurance.csv')

In [3]:
dtale.show(data)

Данные содержат 5000 наблюдений и 5 признаков. Целевой признак - количество страховых выплат за последние 5 лет.

- **Пол** (int) содержит два значения - 1 и 0 (мужской и женский пол). Данные делятся по этому признаку пополам - 2505 наблюдений женского пола, а 2495 - мужского пола. Пропусков в колонке нет. Аномальные значения отсутствуют.

- **Возраст** (float) - описывает возраст клиентов страховой компании. Средний возраст - 30.9 лет Медиана - 30 лет. Максимальный возраст - 65 лет, минимальный - 18. Самый распространенный - 19 лет. Пропущенных значений нет. Аномалий тоже нет.

- **Зарплата** (float) - описывает доход клиентов. Имеет нормальное распределение. Среднее значение 39916 руб, медиана - 40200 руб. Q3 = 46600  - 25% клиентов получают 46600 руб и выше. Пропусков нет, аномалий нет.

- **Члены семьи** (int) - описывает сколько у застрахованного лица есть членов семьи. Распределение следующее: 1 (1814), 0 (1513), 2 (1071), 3 (439), 4 (124), 5 (32), 6 (7). Больших семей мало. Самая распространенная категория - 1 член семьи. Пропусков нет, аномалий тоже.

- **Страховые выплаты** (int) - сколько страховых выплат клиенту было за послдение 5 лет. Рспределение выплат такое: 0 (4436), 1 (423), 2 (115), 3 (18), 4 (7), 5 (1). Большинству клиентов не платили страховку в последние 5 лет. Пропусков нет, аномалий нет.

Дубликатов в данных нет. Необходимо заменить название колонок и тип колонки **Возраст** на int.

-----



In [4]:
# переименуем колонки / rename columns
data.columns = ['sex', 'age', 'income', 'family_members', 'insurance_payout']
data.head()

,sex,age,income,family_members,insurance_payout
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
# заменим тип колонки age на int
# change age column type to int
data['age'] = data['age'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sex               5000 non-null   int64  
 1   age               5000 non-null   int64  
 2   income            5000 non-null   float64
 3   family_members    5000 non-null   int64  
 4   insurance_payout  5000 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 195.4 KB


## Умножение матриц / Matrices multiplication

Обозначения / Symbols:

- $X$ — матрица признаков (нулевой столбец состоит из единиц) / feature matrix (zero column consists of ones)

- $y$ — вектор целевого признака / vector with target values

- $P$ — матрица, на которую умножаются признаки / matrix used to multiple features

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу) / vectors with linear regression weights (zero element is bias)

Предсказания / Predictions:

$$
a = Xw
$$

Задача обучения / Training task:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения / Equation for training:

$$
w = (X^T X)^{-1} X^T y
$$

**Обоснование / Ratonale:**

Пусть у нас есть матрица признаков $X$ размера $mxn$. Также у нас обратимая матрица $P$ размера $nxn$. Результатом умножения $XP$ будет матрица $X_T$ размера $mxn$. То есть при преобразовании матрицы $X$ в $X_T$ ее размер не поменяется.

Let us have a feature matrix $X$ of size $mxn$. We also have an invertible matrix $P$ of size $nxn$. The result of the multiplication $XP$ is a matrix $X_T$ of size $mxn$. That is, when the matrix $X$ is converted to $X_T$, its size will not change.

- **Пусть / Given**

$$
X_T = XP
$$

- **Тогда задача поиска $w$, минимизирующих $MSE$, принимает вид / Then task of $w$ optimization that minimize $MSE$ takes**

$$
w = ((XP)^T XP)^{-1} (XP)^T y
$$

- **Преобразуем по свойству транспонирования произведения матриц $(AB)^T = B^TA^T$ и свойству ассоциативности умножения $A(BC) = (AB)C$ / **We transform the product of matrices $(AB)^T = B^TA^T$ by the transposition property and the associativity property of multiplication $A(BC) = (AB)C$****

$$
w = (P^TX^TXP)^{-1}P^TX^Ty = (P^T(X^TX)P)^{-1}P^TX^Ty = (P^T)^{-1}P^{-1}(X^TX)^{-1}P^TX^Ty
$$

- **$(P^T)^{-1}P^T = I$ - единичная матрица, сокаращаем и остаемся с / identity matrix leaves us with**

$$
w = P^{-1}(X^TX)^{-1}X^Ty
$$

- **Несложно увидеть, что / Easy to see that $(X^TX)^{-1}X^Ty = w$**

$$
P^{-1}w
$$

- **Перейдем к предсказаниям / Move to predictions**

$$
a = XPP^{-1}w
$$

- **По свойству матриц, любая матрица, умноженная на свою обратную матрицу, дает единичную матрицу $AA^{-1}=I$. По итогу, предсказания остались неизменными. Следовательно, качество модели останется неизменным.** / **By the property of matrices, any matrix multiplied by its inverse matrix gives the identity matrix $AA^{-1}=I$. As a result, the predictions remained unchanged. Therefore, the quality of the model will remain unchanged.**
$$
a = Xw
$$

Продемонстрируем это с помощью небольших матриц. Пусть $X$ будет матрицей признаков, $P$ - обратимой рандомной матрицей, а $y$ - вектором с целевыми значениями.

----

Let's demonstrate this using small matrices. Let $X$ be a feature matrix, $P$ be an invertible random matrix, and $y$ be a vector with target values.

In [6]:
X = np.random.rand(50, 10)

P = np.random.rand(10,10)

y = np.random.rand(50,1)

Получим качество модели на оригинальных признаках ($X$).

Let's get the quality of the model on the original features ($X$).

In [7]:
model = LinearRegression()
model.fit(X, y)
predictions = model.predict(X)
print(r2_score(y, predictions))

0.1483733927238


Получим качество модели на трансформированных признаках ($X_T$).

Let's get the quality of the model on the transformed features ($X_T$).

In [8]:
X_t = X @ P
model.fit(X_t, y)
predictions_t = model.predict(X_t)
print(r2_score(y, predictions_t))

0.14837339272380023


Метрика $R^2$ не отличается.

$R^2$ is not changed

## Алгоритм преобразования / Transformation algorithm

**Алгоритм**

1) Получим признаки и целевой признак из данных.

2) Обучим линейную регрессию с применением кросс-валидации на **незашифрованной** выборке. В качестве метрики возьмем ($R^2$).

3) Преобразуем наши данные с помощью умножения матрицы наших признаков на случайную обратимую матрицу.

4) Обучим линейную регрессию с применением кросс-валидаии на **преобразованных** данных и проверим качество модели ($R^2$).

----

**Algorithm**

1) Get the features and the target feature from the data.

2) Train linear regression using cross-validation on an **unencrypted** sample. We take ($R^2$) as a metric.

3) Let's transform our data by multiplying our feature matrix by a random invertible matrix.

4) Train linear regression using cross-validation on **transformed** data and check the quality of the model ($R^2$).

**Обоснование**

При умножении матрицы с признаками на обратимую квадратную матрицу качество линейной регрессии не меняется, так как остаются такими же. Это было доказано через свойства операций над матрицами и продемонстрировано на небольших случайных матрицах.

---

**Rationale**

When multiplying a matrix with features by an invertible square matrix, the quality of the linear regression does not change, as they remain the same. This has been proven through the properties of matrix operations and demonstrated on small random matrices.

## Проверка алгоритма / Algorithm check

Реализуем вышеописанный алгоритм в коде. Создадим класс LinearRegressionEncrypted, с помощью которого сможем преобразовывать наши данные, а затем обучать и проверять модель на них.

----

We implement the above algorithm in code. Let's create a LinearRegressionEncrypted class with which we can transform our data, and then train and validate the model on them.

1) Получим признаки и целевой признак. / Get the target values and features.

In [10]:
features = data.drop('insurance_payout', axis=1).values
target = data['insurance_payout'].values

print(features.shape)
print(target.shape)

(5000, 4)
(5000,)


2) Проверим качество линейной регрессии на обычных данных. / Check quality of model on original data

In [11]:
model = LinearRegression()

cross_val = cross_validate(model, features, target, scoring='r2', cv=3, return_estimator=True)

print('Метрика качества на обычных данных:', round(cross_val['test_score'].mean(), 2))
print()

for i in range(3):
    print(f'Коэффициенты модели для каждого столбца на этапе кросс-валидации {i}:\n', cross_val['estimator'][i].coef_)
    print()

Метрика качества на обычных данных: 0.42

Коэффициенты модели для каждого столбца на этапе кросс-валидации 0:
 [ 9.29122153e-03  3.53947721e-02 -3.24584981e-08 -9.80961785e-03]

Коэффициенты модели для каждого столбца на этапе кросс-валидации 1:
 [ 9.31420962e-03  3.58969571e-02 -2.02084937e-07 -1.64546140e-02]

Коэффициенты модели для каждого столбца на этапе кросс-валидации 2:
 [ 5.08397777e-03  3.58313648e-02 -2.73176844e-07 -1.45356276e-02]



3) Преобразуем данные умножением матрицы наших признаков на случайную обратимую матрицу. / Transform data multiplying matrix with features by random inverse matrix

In [12]:
t = np.random.rand(4, 4) #квадратная матрица 4х4

features_t = features @ t
features_t

array([[25595.03190371,  7567.15090736,  4402.20136595, 42651.43807714],
       [19611.05784882,  5810.76069589,  3378.97037604, 32684.28920561],
       [10838.11980681,  3214.28219208,  1868.88246997, 18064.03562591],
       ...,
       [17491.96016014,  5164.86171457,  3005.18770982, 29146.58313389],
       [16874.2461279 ,  4985.25256098,  2900.15666604, 28118.02361324],
       [20950.02836199,  6189.07418847,  3601.00074337, 34909.38266369]])

4) Проверим качество модели на преобразованных данных. / Check quality of a model on transformed data.

In [13]:
cross_val_t = cross_validate(model, features_t, target, scoring='r2', cv=3, return_estimator=True)

print('Метрика качества на преобразованных данных:', round(cross_val_t['test_score'].mean(), 2))
print()

for i in range(3):
    print(f'Коэффициенты модели для каждого столбца на этапе кросс-валидации {i}:\n', cross_val_t['estimator'][i].coef_)
    print()

Метрика качества на преобразованных данных: 0.42

Коэффициенты модели для каждого столбца на этапе кросс-валидации 0:
 [ 0.0358041  -0.00332199  0.11672805 -0.0329105 ]

Коэффициенты модели для каждого столбца на этапе кросс-валидации 1:
 [ 0.0392423  -0.01921624  0.15396181 -0.03599672]

Коэффициенты модели для каждого столбца на этапе кросс-валидации 2:
 [ 0.01534616 -0.02219877  0.14918234 -0.02063444]



## Общий вывод / Conclusion

В данном проекте мы разрабатывали алгоритм защиты данных клиентов страховой компании. Целью было преобразовать данные так, чтобы качество линейной регрессии не попоменялось. Это необходимо, чтобы одновременно защитить данные клиентов и использовать их для машинного обучения, чтобы помочь бизнесу.

С помощью свойств матричных операций мы математически доказали, что предсказания не отличаются при обучении модели на незашифрованных и зашифрованных данных. Также, мы показали это практически на небольшой матрице рандомных признаков. Метрика осталась неизменной.

Цель была достигнута с помощью матричных операций. Мы преобразовали данные путем умножения признаков на обратимую матрицу размера $nxn$. При этом коэффициенты линейной регрессии двух моделей отличаются. Метрика в обоих случаях осталась неизменной.

----

In this project, we developed an algorithm for protecting the data of clients of an insurance company. The goal was to transform the data so that the quality of the linear regression does not change. This is necessary to both protect customer data and use it for machine learning to help the business.

Using the properties of matrix operations, we mathematically proved that the predictions do not differ when training a model on unencrypted and encrypted data. Also, we showed this practically on a small matrix of random features. The metric has remained unchanged.

The goal was achieved using matrix operations. We transformed the data by multiplying the features by an $nxn$ reversible matrix. At the same time, the coefficients of linear regression of the two models are different. The metric in both cases remained unchanged.